In [ ]:
import pandas as pd
import numpy as np
import perf
import metrics

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
%matplotlib inline

In [ ]:
0.217744


In [109]:
! g++ -Wall -std=c++11 -O3 src/*.cpp -o ./bin/sgd

In [110]:
!time ./bin/sgd

rmse: (0) 0.947487 0
rmse: (100000) 0.216787 0
shuffling 984405
shuffling 744209
shuffling 1003684
shuffling 1216422
shuffling 483813
shuffling 1238357
shuffling 77849
shuffling 1359406
shuffling 592292
shuffling 629971
shuffling 1189137
shuffling 29569
shuffling 1003273
shuffling 1144484
shuffling 986445
shuffling 1271446
shuffling 764037
shuffling 1452091
shuffling 843083
shuffling 546013
shuffling 529537
shuffling 1314471
shuffling 1280103
shuffling 1032207
shuffling 906683
shuffling 785247
shuffling 477121
shuffling 1205036
shuffling 984981
shuffling 856188
shuffling 1384733
shuffling 1436233
shuffling 1397670
shuffling 1046670
shuffling 1211446
shuffling 892087
shuffling 1008625
shuffling 1325531
shuffling 876424
shuffling 867174
shuffling 1198996
shuffling 1342454
shuffling 633025
shuffling 237988
shuffling 170557
shuffling 287980
shuffling 400469
shuffling 682297
shuffling 801999
shuffling 1454256
shuffling 1170392
shuffling 1296994
shuffling 698802
shuffling 435136
shuffling 10

In [ ]:
df = pd.read_csv("./data/results.csv")
print('mean:', round(np.sum(df.prediction) / np.sum(df.exposure), 6), round(np.sum(df.target) / np.sum(df.exposure), 6))
print('rmse:', round(perf.root_mean_square_error(df.target, df.prediction, df.exposure), 6))
print('gini:', metrics.gini_emblem_fast(df.target, df.prediction, df.exposure))
print('deviance:', np.log(metrics.poisson_deviance(df.target, df.prediction, df.exposure)))
metrics.plot_lift_curve(df.target, df.prediction, df.exposure)

In [ ]:
0.21607

mean: 0.043815 0.044271
rmse: 0.216067
gini: 0.32862359198864044
deviance: 12.322948734266841